15.077/IDS.147 Problem Set 4 (Python Version) <br>
**Name:** Chun-Hei Lam. **ID:** 928931321 <br>
**Declaration:** I pledge that the work submitted for this coursework is my own unassisted work unless stated otherwise. <br>
**Acknowledgement to:** Harry Yu

In [19]:
import numpy as np
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import plotly.express as px

# Rice 12.29 Semiconductor

The performance of a semiconductor depends upon the thickness of a layer of silicon dioxide. In an experiment (Czitrom and Reece, 1997), layer thicknesses were measured at three furnace locations for three types of wafers (virgin wafers, recycled in-house wafers, and recycled wafers from an external source). The data are contained in the file `waferlayers`.

In [18]:
Waferlayers = pd.read_csv("waferlayers.txt")

Conduct a two-way analysis of variance and test for significance of main effects and interactions.

Construct a graph such as that shown in Figure 12.3 of textbook. Does the comparison of layer thicknesses depend on furnace location?

# Adhesive Process
An experiment was conducted using an unreplicated $2^4$ factorial design to determine the effects of the amount of glue `x1`, predrying temperature `x2`, tunnel temperature `x3`, and pressure `x4` on the adhesiveforce obtained in an adhesive process.

In [12]:
Adhesive = pd.read_csv("Adhesive.csv")

**Part (a):** Estimate all the main effects and interactions.

**Part (b):** Plot the estimated effects on a normal probability plot.

**Part (c):** Interpret your results.

# Plutonium
A $2^{8-4}$ fractional factorial design was run to identify sources of plutonium contamination in the radioactivity material analysis of dried shellfish at the National Institute of Standards and Technology (NIST). The data are in the table below. No contamination occurred at runs 1,4, and 9.

In [13]:
Plutonium = pd.read_csv("Plutonium.csv")

The factors and levels are shown in the following table.

| Factor | -1 | +1 |
| :- | :- | :- |
| Glassware | Distilled Water | Soap, acid, stored |
| Reagent | New | Old |
| Sample Prep | Co-precipitation | Electrodeposition |
| Tracer | Stock | Fresh |
| Dissolution | Without | With |
| Hood | B | A |
| Chemistry | Without | With |
| Ashing | Without | With |

**Part (a):** Write down the alias relationships.

**Part (b):** Estimate the main effects.

**Part (c):** Prepare a normal probability plot for the effects and interpret the results.

# Inorganic Impurities

This dataset is from an experiment to investigate inorganic impurities (ash) in paper. Two variables, temperature $T$ in degree Celsius and time $t$ in hours, were studied. The coded predictor variables shown in the following table are
\begin{equation}
x_1 = \frac{T-775}{115}, \quad x_2 = \frac{t-3}{1.5}
\end{equation}
and the response $y$ is impurity percentage times $10^3$.

In [14]:
Impurities = pd.read_csv("Impurities.csv")

**Part (a)** What type of design has been used in the study? Can the design be rotated.

**Part (b)** Fit a quadratic model to the data. Is this model satisfactory?

**Part (c)** If it is important to minimise the ash value, where would you run the process?

# Copper Manufacturing
The copper content of a manufacturing process is measured three times per day and the results are reportedas parts per million.  The values for 25 days are given in the table below.

In [15]:
Copper = pd.read_csv("Copper.csv")

**Part (a)** Using all the data, find trial control limits for $\bar{X}$ and $R$ charts, construct the chart, and plot the data. Is the process in statistical control.

**Part (b)** If necessary, revise the control limits computed in part (a), assuming any samples that plot outside the control limits can be eliminated. Continue to eliminate points outside the control limits and revise, until all points plot between control limits.

**Part (c)** Given that the specifications are at $6.0 \pm 0.5$, estimate $C_p$ and $C_{pk}$ for the in-control process and interpret these ratios.